In [1]:
import tensorflow as tf

In [2]:
daily_sales_number = [21, 22, -108, 31, -1, 32, 34, 31]

In [3]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_number)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [4]:
for sales in tf_dataset.take(5):
	print(sales.numpy())

21
22
-108
31
-1


In [5]:
tf_dataset = tf_dataset.filter(lambda x:x>0)
for sales in tf_dataset:
	print(sales.numpy())

21
22
31
32
34
31


In [6]:
tf_dataset = tf_dataset.map(lambda x: x*72)
for sales in tf_dataset:
	print(sales.numpy())

1512
1584
2232
2304
2448
2232


In [7]:
tf_dataet = tf_dataset.shuffle(3)
for sales in tf_dataset:
	print(sales.numpy())

1512
1584
2232
2304
2448
2232


In [8]:
for sales_batch in tf_dataset.batch(4):
	print(sales_batch.numpy())

[1512 1584 2232 2304]
[2448 2232]


In [10]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_number)

In [11]:
tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
	print(sales)

[1512 1584]
[2304 2232]
[2232 2448]


# Pakai image

In [14]:
images_ds = tf.data.Dataset.list_files('./datasets/images/*/*', shuffle=False)

In [15]:
image_count = len(images_ds)
image_count

6

In [16]:
type(images_ds)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [17]:
for file in images_ds.take(3):
	print(file.numpy())

b'.\\datasets\\images\\cat\\01.jpeg'
b'.\\datasets\\images\\cat\\02.jpg'
b'.\\datasets\\images\\cat\\03.jpg'


In [18]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
	print(file.numpy())

b'.\\datasets\\images\\cat\\03.jpg'
b'.\\datasets\\images\\dog\\01.jpg'
b'.\\datasets\\images\\cat\\02.jpg'


In [19]:
class_name = ["cat", "dog"]

In [20]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [21]:
len(train_ds)

4

In [22]:
len(test_ds)

2

In [23]:
def get_label(file_path):
	import os
	parts = tf.strings.split(file_path, os.path.sep)
	return parts[-2]

In [27]:
def process_image(file_path):
	label = get_label(file_path)
	img = tf.io.read_file(file_path)
	img = tf.image.decode_jpeg(img)
	img = tf.image.resize(img, [128, 128])
	return img, label

In [32]:
import os
parts = tf.strings.split("./datasets/images/cat/01.jpeg", os.path.sep)
parts.numpy()

array([b'./datasets/images/cat/01.jpeg'], dtype=object)

In [33]:
img, label = process_image("./datasets/images/cat/01.jpeg")

In [34]:

train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [36]:
for image, label in train_ds.take(3):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[ 43.         78.         48.       ]
  [ 43.         78.         48.       ]
  [ 43.         80.         49.       ]
  ...
  [  6.2730103  30.27301    17.27301  ]
  [  3.1083374  26.108337   16.108337 ]
  [  4.         23.         17.       ]]

 [[ 43.         78.         48.       ]
  [ 43.         78.         48.       ]
  [ 44.81903    79.81903    49.81903  ]
  ...
  [  5.         27.         15.       ]
  [  2.         23.         14.       ]
  [  2.203125   21.203125   15.203125 ]]

 [[ 41.773438   78.         46.160156 ]
  [ 43.         76.77344    46.160156 ]
  [ 43.845886   75.00604    47.845886 ]
  ...
  [  5.3867188  25.386719   14.386719 ]
  [  2.6262817  22.013      14.852844 ]
  [  1.3081665  21.534729   14.921448 ]]

 ...

 [[103.        137.79688    48.40625  ]
  [106.        141.         49.       ]
  [107.6037    143.59412    47.61328  ]
  ...
  [ 99.        141.         43.       ]
  [ 99.        141.         39.       ]
  [101.46918   143.46918    

In [37]:
def scale(image, label):
    return image/255, label

train_ds = train_ds.map(scale)

In [38]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.1764706  0.03137255 0.0627451 ]
****Label:  b'dog'
****Image:  [0.3372549  0.29411766 0.28627452]
****Label:  b'cat'
****Image:  [0.1882353  0.1764706  0.14117648]
****Label:  b'cat'
****Image:  [0.16862746 0.30588236 0.1882353 ]
****Label:  b'dog'
